In [128]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import io

In [129]:
train_data_path = '../data/raw/training_set_VU_DM.csv'
print("Initial test loading:")
chunks_of_data = []
try:
    for chunk in pd.read_csv(train_data_path,chunksize = 50000, parse_dates=['date_time']):
        chunk['search_year'] = chunk['date_time'].dt.year
        chunk['search_month'] = chunk['date_time'].dt.month
        chunk.drop(columns=['date_time'], inplace=True)
        chunks_of_data.append(chunk)
    df = pd.concat(chunks_of_data, ignore_index=True)
    # df_sample = pd.read_csv(train_data_path)
    print(df.head())
    print("\nShape of the sample data:", df.shape)
    print("\nColumn names:")
    print(df.columns.tolist())
    print("\nNull values:")
    print(df.isna().sum())


except FileNotFoundError:
    print(f"File not found at {train_data_path}. Please check the path.")
    

Initial test loading:
   srch_id  site_id  visitor_location_country_id  visitor_hist_starrating  \
0        1       12                          187                      NaN   
1        1       12                          187                      NaN   
2        1       12                          187                      NaN   
3        1       12                          187                      NaN   
4        1       12                          187                      NaN   

   visitor_hist_adr_usd  prop_country_id  prop_id  prop_starrating  \
0                   NaN              219      893                3   
1                   NaN              219    10404                4   
2                   NaN              219    21315                3   
3                   NaN              219    27348                2   
4                   NaN              219    29604                4   

   prop_review_score  prop_brand_bool  ...  comp7_inv  \
0                3.5                1

In [130]:
print("Memoryu usage of the data before downcasting:")

print(df.info(memory_usage='deep'))

Memoryu usage of the data before downcasting:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4958347 entries, 0 to 4958346
Data columns (total 55 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   srch_id                      int64  
 1   site_id                      int64  
 2   visitor_location_country_id  int64  
 3   visitor_hist_starrating      float64
 4   visitor_hist_adr_usd         float64
 5   prop_country_id              int64  
 6   prop_id                      int64  
 7   prop_starrating              int64  
 8   prop_review_score            float64
 9   prop_brand_bool              int64  
 10  prop_location_score1         float64
 11  prop_location_score2         float64
 12  prop_log_historical_price    float64
 13  position                     int64  
 14  price_usd                    float64
 15  promotion_flag               int64  
 16  srch_destination_id          int64  
 17  srch_length_of_stay          int64  
 

In [131]:
# print(df['click_bool'])

for col in df.columns:
    col_type = df[col].dtype

    if col_type == 'int64':
        # Check min/max to see if a smaller int type is sufficient
        c_min = df[col].min()
        c_max = df[col].max()
        if c_min >= np.iinfo(np.int8).min and c_max <= np.iinfo(np.int8).max:
            df[col] = df[col].astype(np.int8)
        elif c_min >= np.iinfo(np.int16).min and c_max <= np.iinfo(np.int16).max:
            df[col] = df[col].astype(np.int16)
        elif c_min >= np.iinfo(np.int32).min and c_max <= np.iinfo(np.int32).max:
            df[col] = df[col].astype(np.int32)
    elif col_type == 'float64':
        # Downcast floats to float32
        df[col] = df[col].astype(np.float32)
    elif col in ['prop_brand_bool', 'promotion_flag', 'srch_saturday_night_bool', 'random_bool', 
                 'click_bool', 'booking_bool'] and col_type != 'int8':
        df[col] = df[col].astype(np.int8)

print("Memory usage of the data after downcasting:")
df.info(memory_usage='deep')

Memory usage of the data after downcasting:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4958347 entries, 0 to 4958346
Data columns (total 55 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   srch_id                      int32  
 1   site_id                      int8   
 2   visitor_location_country_id  int16  
 3   visitor_hist_starrating      float32
 4   visitor_hist_adr_usd         float32
 5   prop_country_id              int16  
 6   prop_id                      int32  
 7   prop_starrating              int8   
 8   prop_review_score            float32
 9   prop_brand_bool              int8   
 10  prop_location_score1         float32
 11  prop_location_score2         float32
 12  prop_log_historical_price    float32
 13  position                     int8   
 14  price_usd                    float32
 15  promotion_flag               int8   
 16  srch_destination_id          int16  
 17  srch_length_of_stay          int8   
 18

In [132]:
#Check for missing values
print("\nMissing Values in the DataFrame:")
print(df.isna().sum())
# --- 1. Calculate and Display Missing Value Percentages---
print("\nMissing Value Percentages (Full Dataset after downcasting):")
missing_percentages = (df.isnull().sum() / len(df)) * 100
# Display only columns that actually have missing values, sorted
print(missing_percentages[missing_percentages > 0].sort_values(ascending=False))




Missing Values in the DataFrame:
srch_id                              0
site_id                              0
visitor_location_country_id          0
visitor_hist_starrating        4706481
visitor_hist_adr_usd           4705359
prop_country_id                      0
prop_id                              0
prop_starrating                      0
prop_review_score                 7364
prop_brand_bool                      0
prop_location_score1                 0
prop_location_score2           1090348
prop_log_historical_price            0
position                             0
price_usd                            0
promotion_flag                       0
srch_destination_id                  0
srch_length_of_stay                  0
srch_booking_window                  0
srch_adults_count                    0
srch_children_count                  0
srch_room_count                      0
srch_saturday_night_bool             0
srch_query_affinity_score      4640941
orig_destination_distance     

In [133]:
#Drop the 'gross_bookings_usd' column as it only is not null if the hotel is booked
df2 = df.copy()
df2.drop('gross_bookings_usd', axis=1, inplace=True)
missing_percentages2 = (df2.isnull().sum() / len(df2)) * 100
print(df.columns.tolist())
print(df2.columns.tolist())
print(missing_percentages2.index.tolist())
print(missing_percentages2[missing_percentages2 > 0].sort_values(ascending=False))




['srch_id', 'site_id', 'visitor_location_country_id', 'visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_country_id', 'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool', 'prop_location_score1', 'prop_location_score2', 'prop_log_historical_price', 'position', 'price_usd', 'promotion_flag', 'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window', 'srch_adults_count', 'srch_children_count', 'srch_room_count', 'srch_saturday_night_bool', 'srch_query_affinity_score', 'orig_destination_distance', 'random_bool', 'comp1_rate', 'comp1_inv', 'comp1_rate_percent_diff', 'comp2_rate', 'comp2_inv', 'comp2_rate_percent_diff', 'comp3_rate', 'comp3_inv', 'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv', 'comp4_rate_percent_diff', 'comp5_rate', 'comp5_inv', 'comp5_rate_percent_diff', 'comp6_rate', 'comp6_inv', 'comp6_rate_percent_diff', 'comp7_rate', 'comp7_inv', 'comp7_rate_percent_diff', 'comp8_rate', 'comp8_inv', 'comp8_rate_percent_diff', 'click_bool', 'gross_

In [134]:
#check correlation to 'booking_bool'
numerical_candidate_features = df2.select_dtypes(include=[np.number]).columns.tolist()
print(numerical_candidate_features)
excluded_features = [
    'srch_id', 'site_id', 'visitor_location_country_id', 'prop_country_id', 'prop_id',
    'prop_brand_bool', 'promotion_flag', 'srch_saturday_night_bool', 'random_bool',
    'position', 
    'click_bool', 'booking_bool']
numerical_features_for_corr = [col for col in numerical_candidate_features if col not in excluded_features]

#threshold picking 
cols_for_initial_corr_check = missing_percentages2[missing_percentages2 < 80].index.tolist()
features_to_corr = [col for col in numerical_features_for_corr if col in cols_for_initial_corr_check]

if features_to_corr: # Check if there are any features left after filtering
    # Calculate correlations with 'booking_bool'
    # .corr() by default uses 'pairwise' method, dropping NA pairs for each calculation.
    correlations_booking = df2[features_to_corr + ['booking_bool']].corr()['booking_bool'].sort_values(ascending=False)
    
    print("\nInitial Correlations with booking_bool (features with <80% missing):")
    print(correlations_booking)

['srch_id', 'site_id', 'visitor_location_country_id', 'visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_country_id', 'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool', 'prop_location_score1', 'prop_location_score2', 'prop_log_historical_price', 'position', 'price_usd', 'promotion_flag', 'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window', 'srch_adults_count', 'srch_children_count', 'srch_room_count', 'srch_saturday_night_bool', 'srch_query_affinity_score', 'orig_destination_distance', 'random_bool', 'comp1_rate', 'comp1_inv', 'comp1_rate_percent_diff', 'comp2_rate', 'comp2_inv', 'comp2_rate_percent_diff', 'comp3_rate', 'comp3_inv', 'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv', 'comp4_rate_percent_diff', 'comp5_rate', 'comp5_inv', 'comp5_rate_percent_diff', 'comp6_rate', 'comp6_inv', 'comp6_rate_percent_diff', 'comp7_rate', 'comp7_inv', 'comp7_rate_percent_diff', 'comp8_rate', 'comp8_inv', 'comp8_rate_percent_diff', 'click_bool', 'bookin

In [147]:
print("\n--- Starting Point for Imputation ---")


df_imputed = df2.copy()


missing_percentages_on_imputed_df = (df_imputed.isnull().sum() / len(df_imputed)) * 100
columns_with_missing = missing_percentages_on_imputed_df[missing_percentages_on_imputed_df > 0].index.tolist()

# Process columns based on missing percentage tiers
for col in columns_with_missing:
    # Get the most up-to-date percentage of missing values for the current column in df_imputed
    # This is important if previous imputations on other columns somehow affected this one (unlikely with simple fillna)
    percentage = (df_imputed[col].isnull().sum() / len(df_imputed)) * 100
    
    print(f"\nProcessing column '{col}' with {percentage:.2f}% missing values...")

    if percentage < 1.0:
        median_val = df_imputed[col].median()
        
        df_imputed[col] = df_imputed[col].fillna(median_val)
        print(f"  Filled with median: {median_val}")
    elif percentage < 33.0:
        median_val = df_imputed[col].median()
        
        df_imputed[col] = df_imputed[col].fillna(median_val)
        print(f"  Filled with median: {median_val}")
    else:
        
        df_imputed[f'{col}_is_missing'] = df2[col].isnull().astype(np.int8)
        print(f"  Created '{col}_is_missing' flag.")

        # Fill original column with a distinct placeholder
        placeholder_val = -999.0
        
        df_imputed[col] = df_imputed[col].fillna(placeholder_val)
        print(f"  Filled original NaNs with placeholder: {placeholder_val}")
        

    
 


--- Starting Point for Imputation ---

Processing column 'visitor_hist_starrating' with 94.92% missing values...
  Created 'visitor_hist_starrating_is_missing' flag.
  Filled original NaNs with placeholder: -999.0

Processing column 'visitor_hist_adr_usd' with 94.90% missing values...
  Created 'visitor_hist_adr_usd_is_missing' flag.
  Filled original NaNs with placeholder: -999.0

Processing column 'prop_review_score' with 0.15% missing values...
  Filled with median: 4.0

Processing column 'prop_location_score2' with 21.99% missing values...
  Filled with median: 0.0689999982714653

Processing column 'srch_query_affinity_score' with 93.60% missing values...
  Created 'srch_query_affinity_score_is_missing' flag.
  Filled original NaNs with placeholder: -999.0

Processing column 'orig_destination_distance' with 32.43% missing values...
  Filled with median: 386.6000061035156

Processing column 'comp1_rate' with 97.58% missing values...
  Created 'comp1_rate_is_missing' flag.
  Filled 

In [149]:
print(df_imputed.isna().sum().sum())
# print(df_imputed.tail)

0


In [146]:
# Ensure the directory exists
output_dir = '../data/modified/' # As per your screenshot
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

csv_output_path = os.path.join(output_dir, 'df_imputed_train.csv')

try:
    df_imputed.to_csv(csv_output_path, index=False) # index=False is important
    print(f"Successfully saved df_imputed to: {csv_output_path}")
    # You can check the file size if you want:
    # print(f"File size: {os.path.getsize(csv_output_path) / (1024*1024):.2f} MB")
except Exception as e:
    print(f"Error saving to CSV: {e}")

Successfully saved df_imputed to: ../data/modified/df_imputed_train.csv
